# Importing Essential Libraries

In [1]:
import sys, os
import pandas as pd
import bokeh.models as bokeh_models
import bokeh.palettes as bokeh_palettes

sys.path.append(os.path.abspath('../..')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.

from st_visions.st_visualizer import st_visualizer
import st_visions.express as viz_express
import st_visions.geom_helper as viz_helper
import st_visions.providers as viz_providers


os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*" # DEV ENVIROMENT BANDAID FOR VSCODE VISUALIZATIONS, will be edited

# Use-Case #1: Using **CategoricalColorMapper** for a **Single Feature**.

**NOTE:** The Categorical Feature must be in String form

Loading Polygon Dataset


In [2]:
world_airports = pd.read_csv(r'..\..\data\airports_dummy.csv')
world_airports = viz_helper.getGeoDataFrame_v2(world_airports, coordinate_columns=['Longitude', 'Latitude'], crs='epsg:4326')

''' For webmercator, when latitudes tend to 90 deg, northing tend to infinity, so using 89.9999 is not the solution. 
    The recommended area of use of EPSG:3857 is for latitudes between -85 and 85 degrees.
    Thus, we get the locations that reside anywhere but the poles. '''

world_airports = world_airports.loc[world_airports.Latitude.between(-85, 85)].copy()
world_airports = world_airports.loc[world_airports.Altitude >= 0].copy()

In [3]:
st_viz = st_visualizer(limit=5000)
st_viz.set_data(world_airports, sp_columns=['Longitude', 'Latitude'])

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='fixed', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

tooltips = [('Name','@Name'), ('City Country','@City_Country'), ('IATA','@IATA'), ('Location','(@Longitude, @Latitude, @Altitude)'), ('Timezone', '@Timezone.1')]

st_viz.add_categorical_colormap('Category20b', 'City_Country')
_ = st_viz.add_marker(marker='circle', fill_color=st_viz.cmap, line_color=st_viz.cmap, alpha=1, legend_group='City_Country')

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True)

Loading BokehJS ...

It should be noted that in order to effectively vizualize the  integer-categorigal feature ``mmsi``, its values require to be casted into String format prior to entering the visualization pipeline.

In [14]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

st_viz.add_categorical_colormap(palette=bokeh_palettes.Viridis256, categorical_name='vessel_id')
circ = st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')


Loading BokehJS ...

In order to colormap numeric features like ``speed``, LinearColorMapper should be used

In [13]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='fixed', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

st_viz.add_numerical_colormap('Viridis256', 'speed', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
_ = st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'Speed Heatmap')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True)



Loading BokehJS ...

# Use-Case #2 Colormap & Filter Interactions

#### Numerical Colormap Example

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

st_viz.add_numerical_colormap('Viridis256', 'speed', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'GPS Locations (Speed Heatmap)')
st_viz.add_temporal_filter(callback_policy='value', temporal_name='t', temporal_unit='ms')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True)

Loading BokehJS ...

#### Categorical Colormap Example

In [24]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

st_viz.add_categorical_colormap(palette=bokeh_palettes.Category20c_20, categorical_name='vessel_id')
st_viz.add_marker(marker='circle', size=10, color=st_viz.cmap, alpha=0.8, fill_alpha=0.7, muted_alpha=0, legend_label=f'Vessel Positions')
st_viz.add_temporal_filter(callback_policy='value', step_ms=1000, temporal_name='t', temporal_unit='ms')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...